<div style='float: right'><img src='pic/eisbahn.png'/></div>
## <div id='eisbahn' />アイスバーン

### 問題
* INから入りOUTにいく１本の線をひきます
* 灰色のマスをアイスバーンとし、必ず通ります
* アイスバーンで曲がってはいけません
* アイスバーンのみ交差可です
* 矢印は必ず通ること

### 変数
* vh：0:L, 1:R (1)
* vv：0:U, 1:D (2)
* vhs (3)
* vvs (4)

### 制約
* vhsをvhで、vvsをvvで表現 (5)
* 矢印は必ず通ること (6)
* 各マスで入る数と出る数が同じこと (7)
* アイスバーンなら横は同じ、縦も同じこと（曲がらない） (8)
* アイスバーンなら線は2以上 (9)
* アイスバーンでないなら線は2以下 (10)
* 線は1つ (11)